<a href="https://colab.research.google.com/github/Feranie/Hierarchical-Classification-Project/blob/main/IRH_Sele%C3%A7%C3%A3o_de_atributos_com_heur%C3%ADstica_RandomRestart-Hill_Climbing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importações necessárias para o funcionamento do algoritmo
import pandas as pd  # Biblioteca para manipulação de dados em formato DataFrame
import numpy as np   # Biblioteca para cálculos numéricos eficientes
import random       # Módulo para geração de números aleatórios
import re          # Módulo para expressões regulares (usado na leitura do ARFF)
from collections import defaultdict  # Dicionário com valores padrão
from typing import List, Set, Tuple  # Tipos para anotações de tipo
import time        # Módulo para medição de tempo de execução
from concurrent.futures import ProcessPoolExecutor  # Para processamento paralelo
import multiprocessing as mp  # Módulo para multiprocessamento

# --- SEÇÃO: Leitura de arquivos ARFF ---
def read_arff_file(file_path):
    """
    Função para ler arquivos no formato ARFF (Attribute-Relation File Format)
    usado pelo Weka e outras ferramentas de mineração de dados
    """
    data = []        # Lista para armazenar os dados de cada instância
    attributes = []  # Lista para armazenar os nomes dos atributos
    current_section = None  # Variável para controlar qual seção do arquivo estamos lendo

    # Abre o arquivo ARFF com codificação UTF-8 para suportar caracteres especiais
    with open(file_path, 'r', encoding='utf-8') as file:
        # Itera sobre cada linha do arquivo
        for line in file:
            line = line.strip()  # Remove espaços em branco no início e fim da linha
            # Pula linhas vazias ou que começam com '%' (comentários no ARFF)
            if not line or line.startswith('%'):
                continue

            # Identifica a seção @relation (nome da relação/dataset)
            if '@relation' in line.lower():
                current_section = 'relation'
            # Identifica a seção @attribute (definição dos atributos)
            elif '@attribute' in line.lower():
                current_section = 'attribute'
                # Usa expressão regular para extrair o nome do atributo
                match = re.match(r'@attribute\s+([^\s]+)\s+.*', line, re.IGNORECASE)
                if match:
                    # Adiciona o nome do atributo à lista de atributos
                    attributes.append(match.group(1))
            # Identifica a seção @data (início dos dados)
            elif '@data' in line.lower():
                current_section = 'data'
            # Se estamos na seção de dados, processa cada linha como uma instância
            elif current_section == 'data':
                # Separa os valores por vírgula (considerando possíveis vírgulas em strings)
                values = re.findall(r'[^,]+(?:,(?=[^,]*$))?', line)
                # Remove aspas e espaços dos valores
                values = [v.strip('" ') for v in values]
                # Verifica se o número de valores corresponde ao número de atributos
                if len(values) == len(attributes):
                    data.append(values)  # Adiciona a instância à lista de dados

    # Retorna um DataFrame do pandas com os dados e nomes das colunas
    return pd.DataFrame(data, columns=attributes)

# --- SEÇÃO: Salvamento de arquivos ARFF ---
def save_to_arff(df, file_path, relation_name="filtered_data"):
    """
    Função para salvar um DataFrame em formato ARFF
    """
    # Abre o arquivo para escrita com codificação UTF-8
    with open(file_path, 'w', encoding='utf-8') as f:
        # Escreve o cabeçalho com o nome da relação
        f.write(f"@relation {relation_name}\n\n")

        # Para cada coluna do DataFrame, define o tipo do atributo
        for column in df.columns:
            unique_values = df[column].unique()  # Obtém valores únicos da coluna
            # Se todos os valores são strings, cria um atributo categórico
            if all(isinstance(val, str) for val in unique_values):
                vals = ','.join(sorted(set(unique_values)))  # Junta os valores únicos
                f.write(f"@attribute {column} {{{vals}}}\n")
            else:
                # Caso contrário, define como atributo numérico
                f.write(f"@attribute {column} numeric\n")

        # Escreve o marcador de início dos dados
        f.write("\n@data\n")
        # Escreve cada linha do DataFrame como uma instância
        for _, row in df.iterrows():
            f.write(",".join(map(str, row)) + "\n")

# --- SEÇÃO: Classe principal para cálculo do IRH (Taxa de Inconsistência Hierárquica) ---
class IRHCalculator:
    """
    Calculadora otimizada para o IRH (Inconsistency Rate Hierarchical)
    usado em problemas de classificação hierárquica
    """
    def __init__(self, data: pd.DataFrame, weight_mode='equal'):
        """
        Inicializa o calculador de IRH
        """
        self.data = data  # Armazena o DataFrame com os dados
        self.weight_mode = weight_mode  # Modo de cálculo dos pesos ('equal', 'softmax', 'hybrid')
        self.h = self._get_number_of_levels()  # Calcula o número de níveis na hierarquia
        self.weights = self._calculate_weights()  # Calcula os pesos para cada nível
        # Cache para evitar recalcular patterns já processados (otimização)
        self._pattern_cache = {}
        # Pré-calcula os patterns de classe para cada nível (otimização)
        self._precompute_class_patterns()

    def _get_number_of_levels(self) -> int:
        """
        Determina o número máximo de níveis na hierarquia de classes
        """
        # Conta o número de pontos em cada classe e pega o máximo
        return max(len(str(c).split('.')) for c in self.data['class'])

    def _precompute_class_patterns(self):
        """
        Pré-calcula os patterns de classe para cada nível hierárquico
        Esta otimização evita recalcular os patterns a cada chamada
        """
        self.class_patterns_by_level = {}  # Dicionário para armazenar patterns por nível
        # Para cada nível da hierarquia
        for level in range(1, self.h + 1):
            class_patterns = []  # Lista para armazenar patterns deste nível
            # Para cada valor de classe nos dados
            for class_val in self.data['class']:
                class_parts = str(class_val).split('.')  # Separa por pontos
                # Se há partes suficientes para este nível
                if len(class_parts) >= level:
                    # Pega apenas as primeiras 'level' partes
                    class_patterns.append('.'.join(class_parts[:level]))
                else:
                    # Se não há partes suficientes, usa a classe completa
                    class_patterns.append(str(class_val))
            # Armazena os patterns deste nível
            self.class_patterns_by_level[level] = class_patterns

    def _calculate_weights(self):
        """
        Calcula os pesos para cada nível da hierarquia
        """
        if self.weight_mode == 'equal':
            # Pesos iguais para todos os níveis
            return [1/self.h for _ in range(self.h)]

        # Para outros modos, retorna None (será calculado dinamicamente)
        return None

    def _calculate_level_inconsistency_rate_optimized(self, attribute_subset: List[str], level: int) -> float:
        """
        Versão otimizada do cálculo da taxa de inconsistência para um nível específico
        """
        # Cria uma chave única para o cache baseada nos atributos e nível
        cache_key = (tuple(sorted(attribute_subset)), level)
        # Se já calculamos este resultado antes, retorna do cache
        if cache_key in self._pattern_cache:
            return self._pattern_cache[cache_key]

        # Obtém apenas os dados dos atributos selecionados como array numpy (mais rápido)
        subset_data = self.data[attribute_subset].values
        # Obtém os rótulos de classe para este nível específico
        class_labels = self.class_patterns_by_level[level]

        # Dicionário aninhado: pattern -> {classe -> contagem}
        pattern_counts = defaultdict(lambda: defaultdict(int))

        # Para cada instância nos dados
        for i, pattern in enumerate(subset_data):
            pattern_tuple = tuple(pattern)  # Converte para tupla (imutável, pode ser chave)
            # Incrementa a contagem desta combinação pattern-classe
            pattern_counts[pattern_tuple][class_labels[i]] += 1

        inconsistency_count = 0  # Contador de instâncias inconsistentes
        # Para cada pattern único encontrado
        for class_counts in pattern_counts.values():
            total_pattern_count = sum(class_counts.values())  # Total de instâncias com este pattern
            max_class_count = max(class_counts.values())      # Maior contagem de uma classe neste pattern
            # Instâncias inconsistentes = total - maior classe (instâncias que não pertencem à classe majoritária)
            inconsistency_count += total_pattern_count - max_class_count

        # Calcula a taxa de inconsistência (proporção de instâncias inconsistentes)
        result = inconsistency_count / len(self.data) if len(self.data) > 0 else float('inf')
        # Armazena no cache para uso futuro
        self._pattern_cache[cache_key] = result
        return result

    def calculate_irh(self, attribute_subset: List[str]) -> float:
        """
        Calcula o IRH (Taxa de Inconsistência Hierárquica) para um subconjunto de atributos
        """
        # Se não há atributos, retorna infinito (pior caso possível)
        if not attribute_subset:
            return float('inf')

        # Calcula a taxa de inconsistência para cada nível da hierarquia
        ir_levels = [
            self._calculate_level_inconsistency_rate_optimized(attribute_subset, level)
            for level in range(1, self.h + 1)
        ]

        # Calcula os pesos baseado no modo selecionado
        if self.weight_mode == 'equal':
            weights = self.weights  # Usa pesos pré-calculados (iguais)
        elif self.weight_mode == 'softmax':
            weights = self._softmax(ir_levels, T=2.0)  # Pesos baseados em softmax
        elif self.weight_mode == 'hybrid':
            # Combina pesos iguais com softmax
            equal = [1/self.h for _ in range(self.h)]
            soft = self._softmax(ir_levels, T=2.0)
            alpha = 0.5  # Fator de combinação
            weights = [(1 - alpha) * e + alpha * s for e, s in zip(equal, soft)]
        else:
            raise ValueError("weight_mode deve ser 'equal', 'softmax' ou 'hybrid'")

        # Retorna a soma ponderada das taxas de inconsistência de todos os níveis
        return sum(w * ir for w, ir in zip(weights, ir_levels))

    def _softmax(self, x: List[float], T: float = 2.0) -> List[float]:
        """
        Calcula softmax dos valores para gerar pesos adaptativos
        T é a temperatura que controla a suavidade da distribuição
        """
        x = np.array(x)  # Converte para array numpy
        e_x = np.exp(-x / T)  # Exponencial negativa (valores menores de IR têm peso maior)
        return (e_x / e_x.sum()).tolist()  # Normaliza para somar 1

# --- SEÇÃO: Classe principal do algoritmo Hill Climbing ---
class RandomRestartHillClimbingFeatureSelector:
    """
    Implementação otimizada do algoritmo Random Restart Hill Climbing
    para seleção de características em problemas de classificação hierárquica
    """
    def __init__(self, data, weight_mode='equal', max_iterations=100, restarts=10,
                 early_stopping_patience=20, min_improvement=1e-6):
        """
        Inicializa o seletor de características
        """
        self.data = data  # Dataset completo
        # Lista de características (todas as colunas exceto 'class')
        self.features = [col for col in data.columns if col != 'class']
        # Inicializa o calculador de IRH
        self.irh_calculator = IRHCalculator(data, weight_mode)
        self.max_iterations = max_iterations  # Número máximo de iterações por restart
        self.restarts = restarts  # Número de reinicializações aleatórias
        self.early_stopping_patience = early_stopping_patience  # Paciência para parada precoce
        self.min_improvement = min_improvement  # Melhoria mínima considerada significativa
        random.seed(42)  # Fixa a semente para reprodutibilidade

    def fitness(self, feature_subset):
        """
        Função de fitness - quanto menor o IRH, melhor o subconjunto
        """
        return self.irh_calculator.calculate_irh(feature_subset)

    def get_neighbors_incremental(self, current_subset: List[str]) -> List[Tuple[List[str], str]]:
        """
        Gera todos os vizinhos possíveis do subconjunto atual
        Retorna tuplas (vizinho, tipo_de_ação) para tracking
        """
        neighbors = []  # Lista para armazenar vizinhos
        current_set = set(current_subset)  # Converte para set para busca eficiente

        # OPERAÇÃO: Adicionar um atributo
        # Prioriza adições pois frequentemente levam a melhorias
        for f in self.features:
            if f not in current_set:  # Se o atributo não está no subconjunto atual
                # Cria vizinho adicionando este atributo
                neighbors.append((current_subset + [f], 'add'))

        # OPERAÇÃO: Remover um atributo
        # Só permite remoção se sobrar pelo menos 1 atributo
        if len(current_subset) > 1:
            for f in current_subset:
                # Cria vizinho removendo este atributo
                neighbors.append(([x for x in current_subset if x != f], 'remove'))

        return neighbors

    def hill_climb_optimized(self, initial_subset):
        """
        Implementação otimizada do algoritmo Hill Climbing
        """
        current_subset = initial_subset  # Subconjunto atual
        current_score = self.fitness(current_subset)  # Score atual
        no_improvement_count = 0  # Contador para parada precoce

        print(f"  Início hill climb com {len(initial_subset)} atributos, IRH inicial: {current_score:.6f}")

        # Loop principal do Hill Climbing
        for iteration in range(self.max_iterations):
            # Gera todos os vizinhos possíveis
            neighbors = self.get_neighbors_incremental(current_subset)
            best_neighbor = current_subset  # Melhor vizinho encontrado
            best_score = current_score      # Melhor score encontrado
            best_action = None             # Ação que levou ao melhor vizinho

            # Lista para armazenar melhorias significativas
            improvements = []
            # Avalia cada vizinho
            for neighbor, action in neighbors:
                score = self.fitness(neighbor)  # Calcula IRH do vizinho
                # Se há melhoria significativa (redução no IRH)
                if score < current_score - self.min_improvement:
                    improvements.append((neighbor, score, action))

            # Se encontrou melhorias
            if improvements:
                # Seleciona a melhor melhoria (menor IRH)
                best_neighbor, best_score, best_action = min(improvements, key=lambda x: x[1])
                current_subset = best_neighbor  # Atualiza subconjunto atual
                current_score = best_score      # Atualiza score atual
                no_improvement_count = 0        # Reseta contador de parada precoce

                # Log periódico do progresso
                if iteration % 10 == 0 or best_action:
                    print(f"    Iteração {iteration}: {best_action} -> IRH={current_score:.6f}, tamanho={len(current_subset)}")
            else:
                # Não houve melhoria
                no_improvement_count += 1
                # Se excedeu a paciência, para o algoritmo
                if no_improvement_count >= self.early_stopping_patience:
                    print(f"    Parada precoce após {iteration} iterações (sem melhoria)")
                    break

        return current_subset, current_score

    def run_parallel(self):
        """
        Executa múltiplos restarts do Hill Climbing (versão paralelizada)
        """
        print(f"Iniciando RRHC otimizado com {self.restarts} restarts...")

        best_global_subset = None   # Melhor subconjunto global
        best_global_score = float('inf')  # Melhor score global

        # Executa cada restart sequencialmente
        for restart in range(self.restarts):
            print(f"\n=== Restart {restart + 1}/{self.restarts} ===")
            start_time = time.time()

            # Gera ponto de partida inteligente
            # Tamanho inicial entre 10% e 25% das características
            initial_size = random.randint(max(1, len(self.features) // 10),
                                        min(len(self.features), len(self.features) // 4))
            # Seleciona características aleatoriamente para o ponto inicial
            initial_feature = random.sample(self.features, initial_size)

            # Executa Hill Climbing a partir deste ponto inicial
            subset, score = self.hill_climb_optimized(initial_feature)
            elapsed = time.time() - start_time

            print(f"Restart {restart + 1} finalizado em {elapsed:.2f}s — IRH: {score:.6f}, tamanho: {len(subset)}")

            # Se encontrou uma solução melhor que a atual melhor
            if score < best_global_score:
                best_global_subset = subset
                best_global_score = score
                print(f"*** NOVA MELHOR SOLUÇÃO: {best_global_score:.6f} ***")

        return best_global_subset, best_global_score

    def run(self):
        """
        Método principal para executar a seleção de características
        """
        return self.run_parallel()

# --- SEÇÃO: Funções auxiliares para paralelização ---
def evaluate_restart(args):
    """
    Função auxiliar para avaliar um restart em paralelo
    (usado na versão totalmente paralelizada)
    """
    # Desempacota os argumentos
    data, features, weight_mode, max_iterations, restart_id, seed = args

    # Fixa semente única para este restart
    random.seed(seed + restart_id)

    # Cria seletor para este restart
    selector = OptimizedRandomRestartHillClimbingFeatureSelector(
        data, weight_mode=weight_mode, max_iterations=max_iterations, restarts=1
    )

    # Gera ponto inicial aleatório
    initial_size = random.randint(max(1, len(features) // 10),
                                min(len(features), len(features) // 4))
    initial_feature = random.sample(features, initial_size)

    # Executa Hill Climbing
    subset, score = selector.hill_climb_optimized(initial_feature)

    return subset, score, restart_id

# --- SEÇÃO: Versão com paralelização completa ---
class ParallelRandomRestartHillClimbing:
    """
    Versão completamente paralelizada do Random Restart Hill Climbing
    """
    def __init__(self, data, weight_mode='equal', max_iterations=100, restarts=10):
        """
        Inicializa o seletor paralelizado
        """
        self.data = data
        self.features = [col for col in data.columns if col != 'class']
        self.weight_mode = weight_mode
        self.max_iterations = max_iterations
        self.restarts = restarts

    def run_parallel_full(self):
        """
        Executa todos os restarts em paralelo usando múltiplos processos
        """
        print(f"Iniciando RRHC paralelizado com {self.restarts} restarts em {mp.cpu_count()} processadores...")

        # Prepara argumentos para cada restart
        args_list = [
            (self.data, self.features, self.weight_mode, self.max_iterations, i, 42)
            for i in range(self.restarts)
        ]

        best_global_subset = None
        best_global_score = float('inf')

        # Executa restarts em paralelo usando ProcessPoolExecutor
        with ProcessPoolExecutor(max_workers=min(mp.cpu_count(), self.restarts)) as executor:
            # Mapeia a função evaluate_restart para cada conjunto de argumentos
            results = list(executor.map(evaluate_restart, args_list))

        # Analisa os resultados de todos os restarts
        for subset, score, restart_id in results:
            print(f"Restart {restart_id + 1}: IRH={score:.6f}, tamanho={len(subset)}")
            # Se encontrou melhor solução
            if score < best_global_score:
                best_global_subset = subset
                best_global_score = score

        return best_global_subset, best_global_score

# --- SEÇÃO: Função principal do programa ---
def main():
    """
    Função principal que orquestra todo o processo de seleção de características
    """
    # Define caminhos dos arquivos de entrada e saída
    input_file_path = '/content/EC-PrintsTRA0.arff'  # Arquivo ARFF de entrada
    output_file_path = '/content/EC-PrintsTRA0ptimizedRRHC.arff'  # Arquivo ARFF de saída

    # ETAPA 1: Carregamento dos dados
    print("Carregando dados...")
    start_time = time.time()
    data = read_arff_file(input_file_path)  # Lê o arquivo ARFF
    load_time = time.time() - start_time

    print(f"Carregados {len(data)} instâncias com {len(data.columns)-1} atributos em {load_time:.2f}s")

    # ETAPA 2: Configuração e execução do algoritmo
    print("\n=== VERSÃO OTIMIZADA SEQUENCIAL ===")
    selector = RandomRestartHillClimbingFeatureSelector(
        data,
        weight_mode='hybrid',       # Modo híbrido para cálculo de pesos
        max_iterations=50,          # Máximo de 50 iterações por restart
        restarts=10,               # 10 reinicializações aleatórias
        early_stopping_patience=15, # Para após 15 iterações sem melhoria
        min_improvement=1e-6       # Melhoria mínima de 0.000001
    )

    # Executa o algoritmo e mede o tempo
    start_time = time.time()
    best_subset, best_irh = selector.run()
    total_time = time.time() - start_time

    # ETAPA 3: Apresentação dos resultados
    print(f"\n=== RESULTADOS FINAIS ===")
    print(f"Tempo total: {total_time:.2f}s")
    print(f"Melhores atributos ({len(best_subset)}): {best_subset}")
    print(f"IRH obtido: {best_irh:.6f}")

    # ETAPA 4: Salvamento dos resultados
    # Cria novo dataset apenas com os atributos selecionados + classe
    selected_data = data[best_subset + ['class']]
    # Salva no formato ARFF
    save_to_arff(selected_data, output_file_path, relation_name="Optimized_Data")
    print(f"Resultados salvos em: {output_file_path}")

# Ponto de entrada do programa
if __name__ == "__main__":
    main()  # Executa a função principal

Carregando dados...


FileNotFoundError: [Errno 2] No such file or directory: '/content/EC-PrintsTRA0.arff'